In [28]:
import numpy as np
import pandas as pd

In [29]:
titanic = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/refs/heads/master/titanic.csv')
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


EDA....

In [30]:
df = titanic[['Pclass','Name','Sex','Age','SibSp','Parch','Fare','Embarked','Survived']].copy()
df

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,0
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,1
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,1
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,1
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,0
...,...,...,...,...,...,...,...,...,...
886,2,"Montvila, Rev. Juozas",male,27.0,0,0,13.0000,S,0
887,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,30.0000,S,1
888,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,23.4500,S,0
889,1,"Behr, Mr. Karl Howell",male,26.0,0,0,30.0000,C,1


Imputaciones..

In [31]:
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
df['Age'] = df['Age'].fillna(df['Age'].median())

Feature Engineer...

In [32]:
# df['Title'] = df['Name'].apply(lambda x: x.split(', ')[1].split('.')[0])
# df['Title'] = df['Name'].str.extract("(Mr|Mrs|Miss|Rev)")

df['Title'] = df['Name'].str.extract(', (\w+)\.').fillna('Sin titulo')
df.drop(columns='Name', inplace=True)

<>:4: SyntaxWarning: invalid escape sequence '\w'
<>:4: SyntaxWarning: invalid escape sequence '\w'
/var/folders/lh/mx1_xdkx76s1jt00m2qwl2km0000gn/T/ipykernel_20236/4247869057.py:4: SyntaxWarning: invalid escape sequence '\w'
  df['Title'] = df['Name'].str.extract(', (\w+)\.').fillna('Sin titulo')


In [33]:
df.to_parquet('df_limpio.parquet',index=False)

------

In [34]:
df = pd.read_parquet('df_limpio.parquet')

procesado para ML

In [35]:
df.select_dtypes(include='object').head(1)

,Sex,Embarked,Title
0,male,S,Mr


In [36]:
df['Sex'] = df['Sex'].map({'male':0, 'female':1})

In [37]:
# a = pd.get_dummies(df['Sex'],dtype='int')

# df2 = df.copy()
# pd.concat([df2, a], axis=1)


In [38]:
# from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder

# pre = OneHotEncoder(handle_unknown='ignore')

# pre.fit(df[['Sex']])

In [39]:
# aa = pd.DataFrame({'Sex':['male','female','Otro']})
# aa

In [40]:
# a = pre.transform(aa).toarray()

In [41]:
# pre.categories_

In [42]:
# pre.feature_names_in_

In [43]:
# pd.DataFrame(a, columns=pre.categories_)

In [44]:
# pre.inverse_transform(a)

In [45]:
df_Emb = pd.get_dummies(df['Embarked'],dtype='int')

df = pd.concat([df, df_Emb], axis=1)
df.drop(columns='Embarked', inplace=True)

In [46]:
from sklearn.preprocessing import OneHotEncoder

pre = OneHotEncoder(handle_unknown='ignore',sparse_output=False)

pre.fit(df[['Title']])

df_Title = pre.transform(df[['Title']])

df_Title =  pd.DataFrame(df_Title, columns=pre.get_feature_names_out(['Title']))

df = pd.concat([df, df_Title], axis=1)

df.drop(columns='Title', inplace=True)

df

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived,C,Q,S,...,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sin titulo,Title_Sir
0,3,0,22.0,1,0,7.2500,0,0,0,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,1,38.0,1,0,71.2833,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3,1,26.0,0,0,7.9250,1,0,0,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,35.0,1,0,53.1000,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,3,0,35.0,0,0,8.0500,0,0,0,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,0,27.0,0,0,13.0000,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
887,1,1,19.0,0,0,30.0000,1,0,0,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
888,3,1,28.0,1,2,23.4500,0,0,0,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
889,1,0,26.0,0,0,30.0000,1,1,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


Division de los datos

In [47]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='Survived'), df['Survived'], 
                                                    test_size=0.2, random_state=42)

In [48]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=2000)

lr.fit(X=X_train, y=y_train)

LogisticRegression(max_iter=2000)

In [49]:
from sklearn.metrics import classification_report, confusion_matrix
pred = lr.predict(X_test)

print(classification_report(y_true=y_test,y_pred=pred,target_names=['Jack','Rose']))

              precision    recall  f1-score   support

        Jack       0.85      0.84      0.84       105
        Rose       0.77      0.78      0.78        74

    accuracy                           0.82       179
   macro avg       0.81      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179



In [50]:
print(confusion_matrix(y_true=y_test,y_pred=pred))

[[88 17]
 [16 58]]


In [51]:
class RegresionLogisticaBootcamp:
    def __init__(self, learning_rate = 0.01, max_iter = 1000):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.pesos = None
        self.sesgo = None
    
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def _binary_cross_entropy(self, y_true, y_pred):

        epsilon = 1e-15 # log(0)
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)

        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    
    def fit(self, df, target):
        X = df.drop(columns=target).values
        y = df[target].values

        n_filas, n_columnas = X.shape
        self.pesos = np.zeros(n_columnas)
        self.sesgo = 0

        for i in range(self.max_iter):

            modelo_lineal = np.dot(X, self.pesos) + self.sesgo

            y_pred = self._sigmoid(modelo_lineal)

            dw = (1 / n_filas) * np.dot(X.T, (y_pred - y))

            db = (1 / n_filas) * np.sum(y_pred - y)

            self.pesos -= self.learning_rate * dw
            self.sesgo -= self.learning_rate * db

            if i % 100 == 0:
                loss = self._binary_cross_entropy(y, y_pred)
                print(f'Iteración {i}: Loss {loss}')

    def predict_proba(self, df):

        if self.pesos is None or self.sesgo is None:
            raise Exception('El modelo no ha sido entrenado')
        
        modelo_lineal = np.dot(df.values, self.pesos) + self.sesgo

        return self._sigmoid(modelo_lineal)
                
    
    def predict(self, df, threshold = 0.5):

        if self.pesos is None or self.sesgo is None:
            raise Exception('El modelo no ha sido entrenado')
        
        probabilidad = self.predict_proba(df)

        return (probabilidad >= threshold).astype(int)


In [52]:
class Skmodel:
    def __init__(self):
        pass

    def split(self):
        pass

    def preprocess(self):
        pass
    
    def train(self):
        pass

    def predict(self):
        pass

    def evaluate(self):
        pass
    
    def save(self):
        pass


-------------